In [ ]:
from decimal import Decimal, getcontext

def pca_colorize_2(features, output_shape, pca):
    # Aplicar PCA como antes
    inverted = False
    remove = True
    rgb = pca.transform(features)
    rgb = min_max_scale(rgb)
    rgb = rgb.reshape(output_shape + (3,))

    # Set the precision you need
    getcontext().prec = 10

    mean_color = np.mean(rgb, axis=(0, 1))
    print(mean_color)
    dominant_color = np.argmax(mean_color)
    print(dominant_color)
    # Convert numpy.float32 to native Python float, then to Decimal
    mean_color_0 = Decimal(float(mean_color[0]))
    mean_color_2 = Decimal(float(mean_color[2]))

    # Calculate the difference using Decimal
    diff =  mean_color_2-mean_color_0

    # Calculate the difference using Decimal
    diff_2 =  mean_color_0 - mean_color_2

    # Aplicar umbral de Otsu al color dominante
    thresh = threshold_otsu(rgb[:, :, 0])
    rgb_mask = (rgb[:, :, 0] > thresh)*1
    # Invertir la máscara si el color dominante no es el primero
    if dominant_color == 2 and diff > Decimal('1.8e-7'):
        print("invertir")
        inverted = True
        rgb_mask = 1 - rgb_mask

    elif dominant_color == 2 and diff <= Decimal('1.8e-7'):
        remove = False
    print(diff_2)
    if dominant_color == 0 and diff_2 <= Decimal('9e-8'):
        remove = False
    rgb[:, :, 0] *= rgb_mask
    rgb[:, :, 1] *= rgb_mask
    rgb[:, :, 2] *= rgb_mask
    rgb = min_max_scale(rgb)

    return rgb, inverted, remove

import os
import torch
import numpy as np
from tqdm import tqdm
from skimage import io
from skimage.transform import resize
from skimage import morphology
from skimage import measure
from skimage.filters import threshold_otsu
from sklearn.decomposition import PCA

from skimage import morphology, measure

def foreground_mask_2(attention_rgb, inverted, remove, use_bbox=True):
    # Crear la máscara básica
    attention_mask = attention_rgb.mean(axis=-1) > 0
    attention_mask = morphology.binary_dilation(attention_mask)
    if remove == False:
        print("no remove")
        # Crear una matriz de unos con las mismas dimensiones que attention_mask
        return np.ones_like(attention_mask)

    if use_bbox:
        attention_labeled = measure.label(attention_mask)
        regions = measure.regionprops(attention_labeled)
        '''
        if inverted:
            # Crear una máscara invertida donde se mantenga todo excepto las bounding boxes
            inverted_mask = np.ones_like(attention_mask, dtype=bool)
            for props in regions:
                ymin, xmin, ymax, xmax = props.bbox
                inverted_mask[ymin:ymax, xmin:xmax] = False
            return inverted_mask
        
        else:
        '''
        # Proceso normal: expandir la máscara dentro de los bounding boxes
        for props in regions:
            ymin, xmin, ymax, xmax = props.bbox
            attention_mask[ymin:ymax, xmin:xmax] = True
    
    return attention_mask

In [ ]:
from sklearn.metrics import pairwise_distances
from skimage.filters import threshold_otsu
from sklearn.decomposition import PCA
from dinov2_feature_extraction import min_max_scale, get_dense_descriptor, foreground_mask

def pca_colorize_2_old(features, output_shape, clusters, remove_bg=False):
    pca = PCA(n_components=3)
    pca.fit(features)
    pca_features = pca.transform(features)
    pca_features = min_max_scale(pca_features)
    pca_image = pca_features.reshape(output_shape + (3,))

    if remove_bg:
        # Calcular las distancias de los canales PCA a los centroides de los clusters
        distances = pairwise_distances(pca_features.reshape(-1, 3), clusters)
        sum_of_distances = distances.sum(axis=1)
        
        # Identificar el canal con la menor suma de distancias a los centroides
        channel_distances = sum_of_distances.reshape(output_shape + (3,)).sum(axis=(0, 1))
        selected_channel = np.argmin(channel_distances)
        
        # Aplicar Otsu al canal seleccionado
        thresh = threshold_otsu(pca_image[:, :, selected_channel])
        background_mask = pca_image[:, :, selected_channel] <= thresh
        
        # Crear una máscara final y aplicarla a todos los canales
        for c in range(3):
            pca_image[background_mask, c] = 0
        
        # Reescalar los valores de la imagen resultante
        pca_image = min_max_scale(pca_image)

    return pca_image

from sklearn.metrics import pairwise_distances
from skimage.filters import threshold_otsu
from sklearn.decomposition import PCA
from dinov2_feature_extraction import min_max_scale

from sklearn.metrics import pairwise_distances


def pca_colorize_2_old_old(features, output_shape, clusters, remove_bg=False):
    # Aplica PCA para reducir la dimensionalidad a 3 componentes
    pca = PCA(n_components=3)
    pca.fit(features)
    pca_components = pca.transform(features)
    pca_components = min_max_scale(pca_components)
    pca_image = pca_components.reshape(output_shape + (3,))

    if remove_bg:
        # Calcula la distancia entre cada punto de características y los centroides de los clusters
        distances = pairwise_distances(pca_components, clusters)
        
        # Suma las distancias por componente
        summed_distances_per_component = np.sum(distances, axis=0)
        
        # Selecciona el componente con la menor suma de distancias a los centroides
        selected_component = np.argmin(summed_distances_per_component % distances.shape[0])
        
        # Aplica el umbral de Otsu al canal seleccionado
        thresh = threshold_otsu(pca_image[:, :, selected_component])
        mask = pca_image[:, :, selected_component] > thresh
        
        # Utiliza la máscara para "borrar" el fondo en todos los canales
        for i in range(3):
            pca_image[mask, i] = 0  # Coloca ceros donde la máscara es verdadera
        
        # Normaliza la imagen resultante
        pca_image = min_max_scale(pca_image)

    return pca_image

In [ ]:
from sklearn.metrics import pairwise_distances



from sklearn.metrics import pairwise_distances
from skimage.filters import threshold_otsu
from sklearn.decomposition import PCA
from dinov2_feature_extraction import min_max_scale

from sklearn.metrics import pairwise_distances


def pca_colorize_2_old_old(features, output_shape, clusters, remove_bg=False):
    # Aplica PCA para reducir la dimensionalidad a 3 componentes
    pca = PCA(n_components=3)
    pca.fit(features)
    pca_components = pca.transform(features)
    pca_components = min_max_scale(pca_components)
    pca_image = pca_components.reshape(output_shape + (3,))

    if remove_bg:
        # Calcula la distancia entre cada punto de características y los centroides de los clusters
        distances = pairwise_distances(pca_components, clusters)
        
        # Suma las distancias por componente
        summed_distances_per_component = np.sum(distances, axis=0)
        
        # Selecciona el componente con la menor suma de distancias a los centroides
        selected_component = np.argmin(summed_distances_per_component % distances.shape[0])
        
        # Aplica el umbral de Otsu al canal seleccionado
        thresh = threshold_otsu(pca_image[:, :, selected_component])
        mask = pca_image[:, :, selected_component] > thresh
        
        # Utiliza la máscara para "borrar" el fondo en todos los canales
        for i in range(3):
            pca_image[mask, i] = 0  # Coloca ceros donde la máscara es verdadera
        
        # Normaliza la imagen resultante
        pca_image = min_max_scale(pca_image)

    return pca_image

def pca_colorize_2_old_old_old(features, output_shape, clusters, remove_bg=False):
    # Aplica PCA para reducir la dimensionalidad a 3 componentes
    pca = PCA(n_components=3)
    pca.fit(features)
    pca_components = pca.transform(features)
    pca_components = min_max_scale(pca_components)
    rgb = pca_components.reshape(output_shape + (3,))

    if remove_bg:
        # Calcula la distancia entre cada punto de características y los centroides de los clusters
        distances = pairwise_distances(pca_components, clusters)
        
        # Suma las distancias para cada componente de PCA
        summed_distances = np.sum(distances, axis=1)

        # Reestructura la suma de distancias para que coincida con el formato de los componentes de PCA
        summed_distances_per_component = summed_distances.reshape(-1, 3).sum(axis=0)

        # Selecciona el componente con la menor suma de distancias a los centroides
        selected_component = np.argmin(summed_distances_per_component)

        # Determina el umbral usando el método de Otsu en el componente seleccionado
        thresh = threshold_otsu(rgb[:, :, selected_component])

        
        # Aplica la umbralización al canal seleccionado y crea una máscara
        if selected_component == 0:
            rgb_mask = (rgb[:, :, 0] > thresh) * 1
        elif selected_component == 1:
            rgb_mask = (rgb[:, :, 1] > thresh) * 1
        else: # selected_component == 2
            rgb_mask = (rgb[:, :, 2] > thresh) * 1

        
        # Aplica la máscara a todos los canales
        for c in range(3):
            rgb[:, :, c] *= rgb_mask
        
        # Reescala los valores de la imagen
        rgb = min_max_scale(rgb)

    return rgb


from sklearn.metrics import pairwise_distances




from sklearn.metrics import pairwise_distances

def pca_colorize_2_asda(features, output_shape, centroids):
    # Realiza PCA en las características
    pca = PCA(n_components=3)
    pca.fit(features)
    rgb = pca.transform(features)
    rgb = min_max_scale(rgb)
    rgb = rgb.reshape(output_shape + (3,))
    
    # Calcula la máscara de umbral
    thresh = threshold_otsu(rgb[:, :, 0])
    mask = (rgb[:, :, 0] > thresh) * 1

    # Aplica la máscara para obtener la imagen sin fondo
    masked_rgb = rgb.copy()
    for i in range(3):  # Aplica la máscara a cada canal
        masked_rgb[:, :, i] *= mask
    
    # Calcula la distancia de la imagen sin fondo a los centroides
    dist_masked_rgb = pairwise_distances(masked_rgb.reshape(-1, 3), centroids).min(axis=1).mean()
    
    # Calcula la distancia de la máscara a los centroides
    # Esto implica que la máscara es una imagen donde solo el fondo está presente
    mask_rgb = rgb.copy()
    for i in range(3):  # Invierte la máscara para el fondo
        mask_rgb[:, :, i] *= (1 - mask)
        
    dist_mask = pairwise_distances(mask_rgb.reshape(-1, 3), centroids).min(axis=1).mean()

    print("DISNTACIA DE LA MÁSCARA: ", dist_mask)
    print("DISNTACIA DE imagen-mascara : ", dist_masked_rgb)
    # Compara las distancias y devuelve la representación que esté más cerca a los centroides
    if dist_masked_rgb <  dist_mask :
        # Devuelve la máscara si la máscara está más cerca de los centroides
        return mask
    else:
        # Devuelve la imagen sin fondo si está más cerca de los centroides
        return masked_rgb
    
def pca_colorize_2_dsad(features, output_shape):
    # Aplicar PCA como antes
    pca = PCA(n_components=3)
    pca.fit(features)
    rgb = pca.transform(features)
    rgb = min_max_scale(rgb)
    rgb = rgb.reshape(output_shape + (3,))

    # Determinar si la imagen es mayoritariamente texto o fondo
    mean_color = np.mean(rgb, axis=(0, 1))
    dominant_color = np.argmax(mean_color)

    # Aplicar umbral de Otsu al color dominante
    thresh = threshold_otsu(rgb[:, :, dominant_color])
    mask = (rgb[:, :, dominant_color] > thresh) if mean_color[dominant_color] < 0.5 else (rgb[:, :, dominant_color] < thresh)

    # Crear la máscara final
    for i in range(3):
        rgb[:, :, i] *= mask

    rgb = min_max_scale(rgb)
    return rgb

In [ ]:
from sklearn.metrics import pairwise_distances

from sklearn.metrics import pairwise_distances


def pca_colorize_2_old_old_old(features, output_shape, clusters, remove_bg=False):
    # Aplica PCA para reducir la dimensionalidad a 3 componentes
    pca = PCA(n_components=3)
    pca.fit(features)
    pca_components = pca.transform(features)
    pca_components = min_max_scale(pca_components)
    rgb = pca_components.reshape(output_shape + (3,))

    if remove_bg:
        # Calcula la distancia entre cada punto de características y los centroides de los clusters
        distances = pairwise_distances(pca_components, clusters)
        
        # Suma las distancias para cada componente de PCA
        summed_distances = np.sum(distances, axis=1)

        # Reestructura la suma de distancias para que coincida con el formato de los componentes de PCA
        summed_distances_per_component = summed_distances.reshape(-1, 3).sum(axis=0)

        # Selecciona el componente con la menor suma de distancias a los centroides
        selected_component = np.argmin(summed_distances_per_component)

        # Determina el umbral usando el método de Otsu en el componente seleccionado
        thresh = threshold_otsu(rgb[:, :, selected_component])

        
        # Aplica la umbralización al canal seleccionado y crea una máscara
        if selected_component == 0:
            rgb_mask = (rgb[:, :, 0] > thresh) * 1
        elif selected_component == 1:
            rgb_mask = (rgb[:, :, 1] > thresh) * 1
        else: # selected_component == 2
            rgb_mask = (rgb[:, :, 2] > thresh) * 1

        
        # Aplica la máscara a todos los canales
        for c in range(3):
            rgb[:, :, c] *= rgb_mask
        
        # Reescala los valores de la imagen
        rgb = min_max_scale(rgb)

    return rgb




def pca_colorize_2_muy_old(features, output_shape, clusters, threshold, remove_bg=False):
    # Aplica PCA para reducir la dimensionalidad a 3 componentes
    pca = PCA(n_components=3)
    pca.fit(features)
    pca_components = pca.transform(features)
    pca_components = min_max_scale(pca_components)
    rgb = pca_components.reshape(output_shape + (3,))

    if remove_bg:
        # Calcula la distancia entre cada punto de características y los centroides de los clusters
        distances = pairwise_distances(pca_components, clusters)

        # Calcula la suma de las distancias a todos los centroides para cada punto
        sum_distances = np.sum(distances, axis=1)

        # Reestructura la matriz de sumas de distancias para que coincida con el formato de los componentes de PCA
        sum_distances = sum_distances.reshape(output_shape)

        # Crea una máscara basada en un umbral de distancia
        distance_mask = sum_distances < threshold

        # Aplica la máscara a todos los canales para eliminar los píxeles cercanos a los centros del cluster
        for c in range(3):
            rgb[distance_mask, c] = 0

        # Reescala los valores de la imagen
        rgb = min_max_scale(rgb)

    return rgb

In [ ]:
from sklearn.metrics import pairwise_distances

def pca_colorize_2_asda(features, output_shape, centroids):
    # Realiza PCA en las características
    pca = PCA(n_components=3)
    pca.fit(features)
    rgb = pca.transform(features)
    rgb = min_max_scale(rgb)
    rgb = rgb.reshape(output_shape + (3,))
    
    # Calcula la máscara de umbral
    thresh = threshold_otsu(rgb[:, :, 0])
    mask = (rgb[:, :, 0] > thresh) * 1

    # Aplica la máscara para obtener la imagen sin fondo
    masked_rgb = rgb.copy()
    for i in range(3):  # Aplica la máscara a cada canal
        masked_rgb[:, :, i] *= mask
    
    # Calcula la distancia de la imagen sin fondo a los centroides
    dist_masked_rgb = pairwise_distances(masked_rgb.reshape(-1, 3), centroids).min(axis=1).mean()
    
    # Calcula la distancia de la máscara a los centroides
    # Esto implica que la máscara es una imagen donde solo el fondo está presente
    mask_rgb = rgb.copy()
    for i in range(3):  # Invierte la máscara para el fondo
        mask_rgb[:, :, i] *= (1 - mask)
        
    dist_mask = pairwise_distances(mask_rgb.reshape(-1, 3), centroids).min(axis=1).mean()

    print("DISNTACIA DE LA MÁSCARA: ", dist_mask)
    print("DISNTACIA DE imagen-mascara : ", dist_masked_rgb)
    # Compara las distancias y devuelve la representación que esté más cerca a los centroides
    if dist_masked_rgb <  dist_mask :
        # Devuelve la máscara si la máscara está más cerca de los centroides
        return mask
    else:
        # Devuelve la imagen sin fondo si está más cerca de los centroides
        return masked_rgb

def pca_colorize_2_dsad(features, output_shape):
    # Aplicar PCA como antes
    pca = PCA(n_components=3)
    pca.fit(features)
    rgb = pca.transform(features)
    rgb = min_max_scale(rgb)
    rgb = rgb.reshape(output_shape + (3,))

    # Determinar si la imagen es mayoritariamente texto o fondo
    mean_color = np.mean(rgb, axis=(0, 1))
    dominant_color = np.argmax(mean_color)

    # Aplicar umbral de Otsu al color dominante
    thresh = threshold_otsu(rgb[:, :, dominant_color])
    mask = (rgb[:, :, dominant_color] > thresh) if mean_color[dominant_color] < 0.5 else (rgb[:, :, dominant_color] < thresh)

    # Crear la máscara final
    for i in range(3):
        rgb[:, :, i] *= mask

    rgb = min_max_scale(rgb)
    return rgb